# Financial Forecasting Model

## Steps
1. Read rows from Google BigQuery
2. Train forecasting model
    - ARIMA
    - LSTM
3. Generate 30 day forecasts
4. Backtest
5. Insert predicted closing prices into Google BigQuery table
6. Visualize in Looker Studio

In [12]:
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os

PROJECT_ID = os.getenv("GCP_PROJECT_ID")
TABLE_ID = "is3107-project-455413.market_data.yf_daily_json"
BIGQUERY_COLUMNS = ["Ticker", "Date", "Open", "High", "Low", "Close", "Volume"]

print(PROJECT_ID)

is3107-project-455413


In [13]:
from google.cloud import bigquery

client = bigquery.Client(PROJECT_ID)
query = f"""
SELECT Ticker, Date, Open, High, Low, Close, Volume
FROM `{TABLE_ID}`
"""

df = client.query(query).to_dataframe()

In [14]:
print(df.head())
print(df.info())
tickers = df["Ticker"].unique()
print(tickers)

  Ticker        Date         Open         High          Low        Close  \
0  ^GSPC  2023-01-03  3853.290039  3878.459961  3794.330078  3824.139893   
1  ^GSPC  2023-01-04  3840.360107  3873.159912  3815.770020  3852.969971   
2  ^GSPC  2023-01-05  3839.739990  3839.739990  3802.419922  3808.100098   
3  ^GSPC  2023-01-06  3823.370117  3906.189941  3809.560059  3895.080078   
4  ^GSPC  2023-01-09  3910.820068  3950.570068  3890.419922  3892.090088   

       Volume  
0  3959140000  
1  4414080000  
2  3893450000  
3  3923560000  
4  4311770000  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3410 entries, 0 to 3409
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Ticker  3410 non-null   object 
 1   Date    3410 non-null   dbdate 
 2   Open    3410 non-null   float64
 3   High    3410 non-null   float64
 4   Low     3410 non-null   float64
 5   Close   3410 non-null   float64
 6   Volume  3410 non-null   Int64  
dtypes: 

### ARIMA Forecasting

#### Model training and Backtest

In [15]:
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')

forecast_records = []
best_orders_by_ticker = {}

for ticker in tickers:
    print(f"PROCESSING {ticker}")
    try:
        ticker_df = df[df["Ticker"] == ticker].sort_values("Date")
        ticker_df.set_index("Date", inplace=True)
        series = ticker_df["Close"]

        if len(series) < 20:
            print(f"Not enough data for {ticker}, skipping.")
            continue

        train_size = int(len(series) * 0.8)
        train, test = series[:train_size], series[train_size:]

        best_aic = float("inf")
        best_order = None
        best_model = None

        for p in range(0, 4):
            for d in range(0, 2):
                for q in range(0, 4):
                    try:
                        model = ARIMA(train, order=(p, d, q))
                        model_fit = model.fit()
                        if model_fit.aic < best_aic:
                            best_aic = model_fit.aic
                            best_order = (p, d, q)
                            best_model = model_fit
                    except:
                        continue

        if best_model:
            best_orders_by_ticker[ticker] = best_order
            forecast = best_model.forecast(len(test))
            forecast.index = test.index
            rmse = np.sqrt(mean_squared_error(test, forecast))
            mae = mean_absolute_error(test, forecast)
            print(f"{ticker} - ARIMA{best_order} | RMSE: {rmse:.2f}, MAE: {mae:.2f}")

            for date in test.index:
                forecast_records.append({
                    "date": date,
                    "ticker": ticker,
                    "predicted_close": forecast.loc[date],
                    "actual_close": test.loc[date],
                    "type": "backtest",
                    "rmse": rmse,
                    "mae": mae,
                    "model": f"ARIMA{best_order}",
                    "training_timestamp": datetime.now()
                })

    except Exception as e:
        print(f"Error processing {ticker}: {e}")

backtest_forecast_df = pd.DataFrame(forecast_records)
backtest_forecast_df.head()

PROCESSING ^GSPC
^GSPC - ARIMA(2, 1, 2) | RMSE: 256.94, MAE: 210.34
PROCESSING ^DJI
^DJI - ARIMA(0, 1, 0) | RMSE: 1818.92, MAE: 1513.91
PROCESSING ^NDX
^NDX - ARIMA(2, 1, 2) | RMSE: 1161.47, MAE: 987.93
PROCESSING BTC-USD
BTC-USD - ARIMA(2, 1, 0) | RMSE: 24302.56, MAE: 22882.24
PROCESSING DOGE-USD
DOGE-USD - ARIMA(1, 0, 3) | RMSE: 0.18, MAE: 0.16


,date,ticker,predicted_close,actual_close,type,rmse,mae,model,training_timestamp
0,2024-10-31,^GSPC,5814.167983,5705.450195,backtest,256.944163,210.340924,"ARIMA(2, 1, 2)",2025-04-22 18:51:53.876596
1,2024-11-01,^GSPC,5814.512634,5728.799805,backtest,256.944163,210.340924,"ARIMA(2, 1, 2)",2025-04-22 18:51:53.876631
2,2024-11-04,^GSPC,5813.652000,5712.689941,backtest,256.944163,210.340924,"ARIMA(2, 1, 2)",2025-04-22 18:51:53.876657
3,2024-11-05,^GSPC,5814.293695,5782.759766,backtest,256.944163,210.340924,"ARIMA(2, 1, 2)",2025-04-22 18:51:53.876681
4,2024-11-06,^GSPC,5814.388133,5929.040039,backtest,256.944163,210.340924,"ARIMA(2, 1, 2)",2025-04-22 18:51:53.876705


#### 7-Day forecast

In [16]:
from datetime import timedelta

forecast_records = []
for ticker in tickers:
    print(f"PREDICTING FUTURE FOR {ticker}")
    try:
        if ticker not in best_orders_by_ticker:
            print(f"No ARIMA order found from backtest for {ticker}, skipping.")
            continue

        order = best_orders_by_ticker[ticker]
        ticker_df = df[df["Ticker"] == ticker].sort_values("Date")
        ticker_df.set_index("Date", inplace=True)
        series = ticker_df["Close"]

        model = ARIMA(series, order=order)
        model_fit = model.fit()
        forecast = model_fit.forecast(steps=7)

        last_date = series.index[-1]
        forecast_dates = pd.date_range(start=last_date + timedelta(days=1), periods=7, freq="D")

        for date, pred in zip(forecast_dates, forecast):
            forecast_records.append({
                "date": date,
                "ticker": ticker,
                "predicted_close": pred,
                "actual_close": None,
                "type": "prediction",
                "rmse": None,
                "mae": None,
                "model": f"ARIMA{order}",
                "training_timestamp": datetime.now()
            })

    except Exception as e:
        print(f"Error predicting future for {ticker}: {e}")


prediction_forecast_df = pd.DataFrame(forecast_records)
print(prediction_forecast_df.tail())

PREDICTING FUTURE FOR ^GSPC
PREDICTING FUTURE FOR ^DJI
PREDICTING FUTURE FOR ^NDX
PREDICTING FUTURE FOR BTC-USD
PREDICTING FUTURE FOR DOGE-USD
         date    ticker  predicted_close actual_close        type  rmse   mae  \
30 2025-04-23  DOGE-USD         0.154496         None  prediction  None  None   
31 2025-04-24  DOGE-USD         0.154370         None  prediction  None  None   
32 2025-04-25  DOGE-USD         0.154245         None  prediction  None  None   
33 2025-04-26  DOGE-USD         0.154120         None  prediction  None  None   
34 2025-04-27  DOGE-USD         0.153997         None  prediction  None  None   

             model         training_timestamp  
30  ARIMA(1, 0, 3) 2025-04-22 18:52:32.522166  
31  ARIMA(1, 0, 3) 2025-04-22 18:52:32.522169  
32  ARIMA(1, 0, 3) 2025-04-22 18:52:32.522171  
33  ARIMA(1, 0, 3) 2025-04-22 18:52:32.522175  
34  ARIMA(1, 0, 3) 2025-04-22 18:52:32.522177  


### XGBoost Regression

#### Model training and Backtest

In [17]:
import pandas as pd
import numpy as np
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor

df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by=["Ticker", "Date"])

# Moving averages for close and volume
for window in [5, 10, 20]:
    df[f"MA_{window}"] = df.groupby("Ticker")["Close"].transform(lambda x: x.rolling(window).mean())
    df[f"Volume_MA_{window}"] = df.groupby("Ticker")["Volume"].transform(lambda x: x.rolling(window).mean())

for lag in range(1, 8):
    df[f"lag_{lag}"] = df.groupby("Ticker")["Close"].shift(lag)

for i in range(1, 8):
    df[f"Close_t+{i}"] = df.groupby("Ticker")["Close"].shift(-i)

df = df.dropna()


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pandas as pd
import numpy as np
from datetime import datetime

forecast_records = []

for ticker in tickers:
    print(f"Training XGBoost for {ticker}")
    ticker_df = df[df["Ticker"] == ticker]

    feature_cols = ['Open', 'High', 'Low', 'Volume',
                    'MA_5', 'MA_10', 'MA_20',
                    'Volume_MA_5', 'Volume_MA_10', 'Volume_MA_20'] + \
                   [f"lag_{i}" for i in range(1, 8)]
    
    target_cols = [f"Close_t+{i}" for i in range(1, 8)]

    X = ticker_df[feature_cols]
    y = ticker_df[target_cols]

    # Split data for backtesting
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Train XGBoost model
    model = MultiOutputRegressor(XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5))
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    all_predictions = []
    all_actuals = []

    test_dates = ticker_df["Date"].iloc[train_size:].reset_index(drop=True)

    for i in range(len(X_test)):
        actuals = y_test.iloc[i].values

        for day in range(7):  # Predict each future day (t+1 to t+7)
            predicted_close = y_pred[i][day]
            
            all_predictions.append(predicted_close)
            all_actuals.append(actuals[day])

            forecast_records.append({
                "date": test_dates[i] + pd.Timedelta(days=day+1),  # t+1 to t+7
                "ticker": ticker,
                "predicted_close": predicted_close,
                "actual_close": actuals[day],
                "type": "backtest",
                "rmse": None,  # Calculated after the loop
                "mae": None,   # Calculated after the loop
                "model": "XGBoost-MultiOut",
                "training_timestamp": datetime.now()
            })

    total_rmse = np.sqrt(mean_squared_error(all_actuals, all_predictions))
    total_mae = mean_absolute_error(all_actuals, all_predictions)

    for record in forecast_records:
        if record["type"] == "backtest":
            record["rmse"] = total_rmse
            record["mae"] = total_mae

    # Predict next 7 days based on the most recent row (for future forecasting)
    X_latest = X.iloc[[-1]]
    y_future_pred = model.predict(X_latest)[0]

    last_date = ticker_df["Date"].max()
    forecast_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=7)

    for i, pred in enumerate(y_future_pred):
        forecast_records.append({
            "date": forecast_dates[i],
            "ticker": ticker,
            "predicted_close": pred,
            "actual_close": None,
            "type": "prediction",
            "rmse": None,
            "mae": None,
            "model": "XGBoost-MultiOut",
            "training_timestamp": datetime.now()
        })

forecast_df = pd.DataFrame(forecast_records)

Training XGBoost for ^GSPC
Training XGBoost for ^DJI
Training XGBoost for ^NDX
Training XGBoost for BTC-USD
Training XGBoost for DOGE-USD


In [19]:
xgboost_prediction_forecast_df = pd.DataFrame(forecast_records)
print(xgboost_prediction_forecast_df.tail())

           date    ticker  predicted_close  actual_close        type  rmse  \
4622 2025-04-16  DOGE-USD         0.153838           NaN  prediction   NaN   
4623 2025-04-17  DOGE-USD         0.154633           NaN  prediction   NaN   
4624 2025-04-18  DOGE-USD         0.148188           NaN  prediction   NaN   
4625 2025-04-19  DOGE-USD         0.144623           NaN  prediction   NaN   
4626 2025-04-20  DOGE-USD         0.159782           NaN  prediction   NaN   

      mae             model         training_timestamp  
4622  NaN  XGBoost-MultiOut 2025-04-22 18:52:38.818356  
4623  NaN  XGBoost-MultiOut 2025-04-22 18:52:38.818364  
4624  NaN  XGBoost-MultiOut 2025-04-22 18:52:38.818371  
4625  NaN  XGBoost-MultiOut 2025-04-22 18:52:38.818378  
4626  NaN  XGBoost-MultiOut 2025-04-22 18:52:38.818385  


In [20]:

from pandas_gbq import to_gbq

DESTINATION_TABLE_ID = "is3107-project-455413.market_data.stock_forecast"
final_forecast_df = pd.concat([backtest_forecast_df, prediction_forecast_df, xgboost_prediction_forecast_df])

final_forecast_df["date"] = pd.to_datetime(final_forecast_df["date"], errors='coerce')
final_forecast_df["training_timestamp"] = pd.to_datetime(final_forecast_df["training_timestamp"], errors='coerce')

final_forecast_df["ticker"] = final_forecast_df["ticker"].astype(str)
final_forecast_df["type"] = final_forecast_df["type"].astype(str)
final_forecast_df["model"] = final_forecast_df["model"].astype(str)

final_forecast_df["predicted_close"] = pd.to_numeric(final_forecast_df["predicted_close"], errors='coerce')
final_forecast_df["actual_close"] = pd.to_numeric(final_forecast_df["actual_close"], errors='coerce')

to_gbq(
    final_forecast_df,
    DESTINATION_TABLE_ID,
    project_id=PROJECT_ID,
    if_exists='replace',
)